## Demonstração - Pytorch

## Um exemplo mais completo

### Carrega os dados e os separa em treino e teste

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"

dfPropaganda = pd.read_csv('Advertising.csv',index_col=0)

y= dfPropaganda.loc[:,'Sales']
X = dfPropaganda.loc[:,['TV','Radio','Newspaper']]

y_tensor = torch.tensor(y.to_numpy())
X_tensor = torch.tensor(X.to_numpy())

X_treino, X_teste, y_treino, y_teste = train_test_split(X_tensor, y_tensor, test_size = 0.30, random_state=5)

print(X_treino.shape)
print(y_treino.shape)

print(X_teste.shape)
print(y_teste.shape)
print(y_teste.ndim)
print(X_treino.dtype)

#Manda para o dispositivo selecionado - no caso, GPU
X_treino = X_treino.float().to(device)
y_treino = y_treino.float().to(device)

#reshape com -1: infere o valor com base no que está sendo passado (passo para sair de 1d e ir para 2d e evitar problemas de cálculos).
y_treino = torch.reshape(y_treino, (-1,1))

X_teste = X_teste.float().to(device)
y_teste = y_teste.float().to(device)
y_teste = torch.reshape(y_teste, (-1,1))

print("\nDepois da conversão para float e do reshape")
print(X_treino.dtype)
print(y_treino.shape)
print(y_teste.shape)
print(y_teste.ndim)

#Já convertemos para float para, pois inicialmente o padrão é double'

torch.Size([140, 3])
torch.Size([140])
torch.Size([60, 3])
torch.Size([60])
1
torch.float64

Depois da conversão para float e do reshape
torch.float32
torch.Size([140, 1])
torch.Size([60, 1])
2


### Implementa uma classe com o modelo

Para definir uma rede neural em PyTorch, criamos uma classe que herda de nn.Module. Definimos as camadas da rede na função __init__ e especificamos como os dados passarão pela rede na função forward. <br>


In [8]:
class Feedforward(torch.nn.Module):
    
        #Esta função é onde você define as camadas totalmente conectadas em sua rede neural
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc4 = torch.nn.Linear(self.hidden_size, 1)
            
        
        #Forward: Especifica como os dados passarão pelo seu modelo
        #x representa nossos dados
        def forward(self, x):
            
            output = self.fc1(x)
            output = F.relu(output)
            
            
            output = self.fc2(output)
            output = F.relu(output)
            
            output = self.fc3(output)
            output = F.relu(output)
            
            output = self.fc4(output)

            return output

### Modelo e critério de otimização
Para acelerar as operações na rede neural, nós a movemos para a GPU, se disponível.

In [9]:
#input = 3 (número de features), e hidden size = 10 (número de neurôneos na camada escondida)
model = Feedforward(3, 10).to(device)
print(model)
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

Feedforward(
  (fc1): Linear(in_features=3, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=1, bias=True)
)


### Treino e teste do modelo

In [10]:
model.eval()
print(X_teste.shape)
y_pred = model(X_teste)
antes_treino = criterion(y_pred, y_teste) 
print('Teste - perda antes do treinamento' , antes_treino.item())


#-----------------Treinamento
model.train()
epoch = 10000

for epoch in range(epoch):
    
    #Explicitamente configura os gradientes em zero
    # optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(X_treino)
    
    # Computa a perda
    loss = criterion(y_pred, y_treino)
    
    print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Propaga os erros (backpropagation) e atualiza os pesos
    loss.backward()
    optimizer.step()

    
    
#-----------------Avaliação
model.eval()

y_pred = model(X_teste)
after_train = criterion(y_pred, y_teste) 
print('Teste - perda depois do treinamento' , after_train.item())    


torch.Size([60, 3])
Teste - perda antes do treinamento 40.96809387207031
Epoch 0: perda treino: 46.03242874145508
Epoch 1: perda treino: 42.60737228393555
Epoch 2: perda treino: 39.47600555419922
Epoch 3: perda treino: 36.547882080078125
Epoch 4: perda treino: 33.8030891418457
Epoch 5: perda treino: 31.251359939575195
Epoch 6: perda treino: 28.90194320678711
Epoch 7: perda treino: 26.776203155517578
Epoch 8: perda treino: 24.898799896240234
Epoch 9: perda treino: 23.300792694091797
Epoch 10: perda treino: 21.999738693237305
Epoch 11: perda treino: 21.032733917236328
Epoch 12: perda treino: 20.440099716186523
Epoch 13: perda treino: 20.2598876953125
Epoch 14: perda treino: 20.518423080444336
Epoch 15: perda treino: 21.246755599975586
Epoch 16: perda treino: 22.466882705688477
Epoch 17: perda treino: 24.193479537963867
Epoch 18: perda treino: 26.4385986328125
Epoch 19: perda treino: 29.191011428833008
Epoch 20: perda treino: 32.403499603271484
Epoch 21: perda treino: 35.99208450317383
Ep